In [2]:
path = '/Users/alvira/Desktop/ml1/kaggle/sales/'
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
import os
import gc
import re
from collections import defaultdict, Counter
import inflect
p = inflect.engine()
from datetime import datetime
import math
import pandas as pd
from datetime import timedelta
dtypes = {'id':'int64', 'item_nbr':'int32', 'store_nbr':'int8','dow':'int8'}

In [2]:
# column_names = ["id", "date", "store_nbr", "item_nbr", "unit_sales","onpromotion"]
dtypes = {'id':'int64', 'item_nbr':'int32', 'store_nbr':'int8'}

train = pd.read_csv(path+'train.csv', usecols=[1,2,3,4], dtype=dtypes, parse_dates=['date'],
                    skiprows=range(1, 101688779) #Skip dates before 2017-01-01
                    )

train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion
# train['dow'] = train['date'].dt.dayofweek

In [3]:
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()

train.set_index(["date", "store_nbr", "item_nbr"], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=["date", "store_nbr", "item_nbr"]
    )
)

del u_dates, u_stores, u_items

In [3]:
def weight_med(input_list):
    dic = {0:1.5, 1:7, 2:7, 3:6, 4:5, 5:4, 6:3, 7:2}
    input_list = np.sort(input_list)
    middle = float(len(input_list))
    if middle%2 != 0:
        return input_list[int(middle/2 - .5)]
    else:
        w1 = dic[np.where(input_list == input_list[int(middle/2)])[0][0]]
        w2 = dic[np.where(input_list == input_list[int(middle/2-1)])[0][0]]
        v1 = input_list[int(middle/2)]*w1
        v2 = input_list[int(middle/2-1)]*w2
        return (v1+v2)/(w1+w2)

In [5]:
# Fill NaNs
train.loc[:, "unit_sales"].fillna(0, inplace=True)
train.reset_index(inplace=True) # reset index and restoring unique columns 
train['dow'] = train['date'].dt.dayofweek

lastdate = train.iloc[train.shape[0]-1].date

In [6]:
#Load test
test = pd.read_csv(path+'test.csv', dtype=dtypes, parse_dates=['date'])
test['dow'] = test['date'].dt.dayofweek
test = test.set_index(['item_nbr', 'store_nbr'])
ltest = test.shape[0]

In [7]:
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw')
ma_dw.reset_index(inplace=True)
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk')
ma_wk.reset_index(inplace=True)
ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais226')

In [8]:
for i in [1,3,7,14,28,56,112]:
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')

del tmp,tmpg

In [9]:
ma_is['mais']=ma_is.apply(lambda x: weight_med(x.values), axis =1)
ma_is.reset_index(inplace=True)

In [10]:
test.reset_index(inplace=True)
test = pd.merge(test, ma_is, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_wk, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])

del ma_is, ma_wk, ma_dw

In [11]:
test['unit_sales'] = test.mais
pos_idx = test['mawk'] > 0
test_pos = test.loc[pos_idx]
test.loc[pos_idx, 'unit_sales'] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']
test.loc[:, "unit_sales"].fillna(0, inplace=True)
test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1) # restoring unit values 

#15% more for promotion items
#By tarobxl: https://www.kaggle.com/tarobxl/overfit-lb-0-532-log-ma
test.loc[test['onpromotion'] == True, 'unit_sales'] = test.loc[test['onpromotion'] == True, 'unit_sales'] * 1.15

test[['id','unit_sales']].to_csv('ma8dwof.csv', index=False, float_format='%.3f')

In [12]:
test_items = pd.unique(test.item_nbr)
train_items = pd.unique(train.item_nbr)
items_only_in_test = set(test_items).difference(set(train_items))

In [13]:
tmp = train[train.date>lastdate-timedelta(15)]
sales_per_item = tmp[['item_nbr','unit_sales']].groupby(['item_nbr'])['unit_sales'].mean().to_frame('spi')
sales_per_item.reset_index(inplace=True)

In [35]:
test2 = pd.merge(test, sales_per_item, how = 'left', on=['item_nbr'])
test2.loc[:, "spi"].fillna(0, inplace=True)
test2['unit_sales'] = np.where(test2['unit_sales'] == 0, test2['spi']/5.0, test2['unit_sales'] )
test2[['id','unit_sales']].to_csv('ma8dwof.csv', index=False, float_format='%.3f')

In [14]:
items = pd.read_csv(path+'items.csv')

In [44]:
sales_per_item.loc[sales_per_item['item_nbr']==test.iloc[2,0],'spi']/2.0

2    0.334366
Name: spi, dtype: float64

In [32]:
test2

,item_nbr,store_nbr,id,date,onpromotion,dow,mais226,mais1,mais3,mais7,mais14,mais28,mais56,mais112,mais,mawk,madw,unit_sales,spi
0,96995,1,125497040,2017-08-16,False,2,0.094532,0.000000,0.000000,0.099021,0.334438,0.295202,0.172356,0.154255,0.124127,0.094875,0.086643,0.120032,0.126618
1,99197,1,125497041,2017-08-16,False,2,0.286959,0.000000,0.000000,0.156945,0.206455,0.280721,0.397257,0.470665,0.240212,0.287493,0.344945,0.334045,0.048794
2,103501,1,125497042,2017-08-16,False,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.334366,0.668732
3,103520,1,125497043,2017-08-16,False,2,0.735661,0.000000,0.231049,0.495105,0.573577,0.726316,0.773160,0.815118,0.643004,0.738038,0.834938,1.069746,0.792965
4,103665,1,125497044,2017-08-16,False,2,1.015079,0.693147,0.462098,0.980990,1.031388,1.026335,0.996363,1.038379,1.004870,1.016829,1.107186,1.986686,0.779424
5,105574,1,125497045,2017-08-16,False,2,1.745741,1.609438,0.998577,1.560437,1.629185,1.699200,1.702692,1.721587,1.661010,1.749076,2.134940,6.594658,1.111210
6,105575,1,125497046,2017-08-16,False,2,2.264586,2.197225,2.123871,2.304131,2.382527,2.306424,2.294462,2.240521,2.278166,2.266248,2.454617,10.793238,1.467086
7,105576,1,125497047,2017-08-16,False,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.698215,1.396430
8,105577,1,125497048,2017-08-16,False,2,0.562736,1.098612,0.597253,0.553029,0.474556,0.517999,0.536972,0.598495,0.557441,0.564541,0.906894,1.448530,0.556577
9,105693,1,125497049,2017-08-16,False,2,0.170306,0.693147,0.693147,0.553029,0.276514,0.276514,0.244519,0.171770,0.276514,0.170838,0.301623,0.629379,0.607257


In [4]:
dtypes = {'id':'int64', 'item_nbr':'int32', 'store_nbr':'int8'}

train = pd.read_csv(path+'train.csv', usecols=[1,2,3,4], dtype=dtypes, parse_dates=['date'],skiprows=range(1, 60000000) )

holiday =  pd.read_csv(path+'holidays_events.csv')
holiday = holiday.loc[holiday['transferred'] == False]
holiday =holiday.rename(columns = {'locale_name':'city'})

train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion
train['dow'] = train['date'].dt.dayofweek

u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
)
del u_dates, u_stores, u_items

train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs
train.reset_index(inplace=True) # reset index and restoring unique columns  
lastdate = train.iloc[train.shape[0]-1].date

#Load test
test = pd.read_csv(path+'test.csv', dtype=dtypes, parse_dates=['date'])
test['dow'] = test['date'].dt.dayofweek
#test['mon'] = test['date'].dt.month

#Days of Week Means
#By tarobxl: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/discussion/42948
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw')
ma_dw.reset_index(inplace=True)
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk')
ma_wk.reset_index(inplace=True)

#Moving Averages
ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais226')
for i in [1,3,7,14,28,56,112]:
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')

del tmp,tmpg

#ma_is['mais']=ma_is.median(axis=1)
ma_is['mais']=ma_is.apply(lambda x: weight_med(x.values), axis =1)
ma_is.reset_index(inplace=True)

test = pd.merge(test, ma_is, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_wk, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])

del ma_is, ma_wk, ma_dw

#Forecasting Test
test['unit_sales'] = test.mais
pos_idx = test['mawk'] > 0
test_pos = test.loc[pos_idx]
test.loc[pos_idx, 'unit_sales'] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']
test.loc[:, "unit_sales"].fillna(0, inplace=True)
test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1) 
test['date']= pd.to_datetime(test['date'])
holiday['date']= pd.to_datetime(holiday['date'])
test1 = pd.merge(test, holiday, how = 'left', on =['date'] )


test1['transferred'].fillna(True, inplace=True)
test1.loc[test1['transferred'] == False, 'unit_sales'] = test1.loc[test1['transferred'] == False, 'unit_sales'] * 1.2
test1.loc[test1['onpromotion'] == True, 'unit_sales'] = test1.loc[test1['onpromotion'] == True, 'unit_sales']*1.5
test1[['id','unit_sales']].to_csv(path+'grocery_3000_holiday.csv.gz', index=False,
                                  float_format='%.3f', compression='gzip')


In [5]:
train = pd.read_csv(path+'train.csv', usecols=[1,2,3,4], dtype=dtypes, parse_dates=['date'],
                    skiprows=range(1, 86672217))

train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion
train['dow'] = train['date'].dt.dayofweek

# creating records for all items, in all markets on all dates
# for correct calculation of daily unit sales averages.
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
)

del u_dates, u_stores, u_items

train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs
train.reset_index(inplace=True) # reset index and restoring unique columns  
lastdate = train.iloc[train.shape[0]-1].date

#Days of Week Means
#By tarobxl: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/discussion/42948
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw')
ma_dw.reset_index(inplace=True)
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk')
ma_wk.reset_index(inplace=True)

#Moving Averages
ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais226')
for i in [112,56,28,14,7,3,1]:
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')

del tmp,tmpg,train

ma_is['mais']=ma_is.apply(lambda x: weight_med(x.values), axis =1)
ma_is.reset_index(inplace=True)

#Load test
test = pd.read_csv(path+'test.csv', dtype=dtypes, parse_dates=['date'])
test['dow'] = test['date'].dt.dayofweek
test = pd.merge(test, ma_is, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_wk, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])

del ma_is, ma_wk, ma_dw

#Forecasting Test
test['unit_sales'] = test.mais 
pos_idx = test['mawk'] > 0
test_pos = test.loc[pos_idx]
test.loc[pos_idx, 'unit_sales'] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']
test.loc[:, "unit_sales"].fillna(0, inplace=True)
test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1) # restoring unit values 

#50% more for promotion items
test.loc[test['onpromotion'] == True, 'unit_sales'] *= 1.5

sub1 = test[['id','unit_sales']]

##-------------
## Second half of code is forked from Nimesh's kernel found here:
## https://www.kaggle.com/nimesh280/ma-forecasting-with-holiday-effect-lb-0-529

sub2 = test1[['id','unit_sales']]

##-------------
## Now we just take the min of the two unit_sales
##-------------

sub1 = sub1.add_suffix('_1')
subnew = pd.concat([sub2, sub1], axis=1, join='inner')
del subnew['id_1']
subnew['unit_sales_min'] = subnew[['unit_sales_1','unit_sales']].min(axis=1)
submin = subnew
del submin['unit_sales_1']
del submin['unit_sales']
submin = submin.rename(columns={'unit_sales_min': 'unit_sales'})

submin.to_csv('submin.csv.gz', index=False,
float_format='%.3f', compression='gzip')
print("done")

done


In [61]:
lgb = pd.read_csv(path+"lgb.csv")
pp = pd.read_csv(path+"ma8dwof.csv")
lgb['unit_sales'] = np.mean([lgb['unit_sales'], pp['unit_sales']], axis =0)
lgb.to_csv(path+"lgb_edit.csv", index=False)

In [60]:
# -*- coding: utf-8 -*-
import pandas as pd
from datetime import timedelta

dtypes = {'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32'}

train = pd.read_csv(path+'train.csv', usecols=[1,2,3,4], dtype=dtypes, parse_dates=['date'],
                    skiprows=range(1, 86672217) #Skip dates before 2016-08-01
                    )

train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion
train['dow'] = train['date'].dt.dayofweek

#Days of Week Means
#By tarobxl: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/discussion/42948
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(
        ['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw').reset_index()
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(
        ['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk').reset_index()

train.drop('dow',1,inplace=True)

# creating records for all items, in all markets on all dates
# for correct calculation of daily unit sales averages.
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
).reset_index()

del u_dates, u_stores, u_items

train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs
lastdate = train.iloc[train.shape[0]-1].date

#Moving Averages
ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(
        ['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais')

for i in [112,56,28,14,7,3,1]:
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')

del tmp,tmpg,train

ma_is['mais']=ma_is.apply(lambda x: weight_med(x.values), axis =1)
ma_is.reset_index(inplace=True)
ma_is.drop(list(ma_is.columns.values)[3:],1,inplace=True)

#Load test
test = pd.read_csv(path+'test.csv', dtype=dtypes, parse_dates=['date'])
test['dow'] = test['date'].dt.dayofweek
test = pd.merge(test, ma_is, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_wk, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])

del ma_is, ma_wk, ma_dw

#Forecasting Test
test['unit_sales'] = test.mais 
pos_idx = test['mawk'] > 0
test_pos = test.loc[pos_idx]
test.loc[pos_idx, 'unit_sales'] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']
test.loc[:, "unit_sales"].fillna(0, inplace=True)
test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1) # restoring unit values 

#50% more for promotion items
test.loc[test['onpromotion'] == True, 'unit_sales'] *= 1.5

test[['id','unit_sales']].to_csv('ma8dwof.csv.gz', index=False, float_format='%.3f', compression='gzip')

In [59]:
test1 = pd.merge(test, holiday, how = 'left', on =['date'] )
test1['transferred'].fillna(True, inplace=True)

test1.loc[test1['transferred'] == False, 'unit_sales'] = test1.loc[test1['transferred'] == False, 'unit_sales'] * 1.2
test1.loc[test['onpromotion'] == True, 'unit_sales'] *= 1.5

# del ma_is, ma_wk, ma_dw

#Forecasting Test
test1.loc[:, "unit_sales"].fillna(0, inplace=True)
test1['unit_sales'] = test1['unit_sales'].apply(pd.np.expm1) # restoring unit values 

#50% more for promotion items


test1[['id','unit_sales']].to_csv('submission.csv.gz', index=False, float_format='%.3f', compression='gzip')